In [2]:
# import libraries
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [3]:
#Import needed libraries
import os.path
import tracktable

from tracktable.core import geomath
from tracktable.domain.terrestrial import TrajectoryPointReader
from tracktable.applications.assemble_trajectories import AssembleTrajectoryFromPoints
from tracktable.render.render_trajectories import render_trajectories

from datetime import datetime, timedelta

tracktable.__version__

'1.6.0'

In [4]:
# read in file of interest
file = '/anvil/projects/tdm/corporate/sandia-trajectory/previous_files/flight/data/raw_data/asdi_2013_10_21.tsv'
data = pd.read_csv(file, sep='\t').dropna()

In [5]:
# set data column names (as they were missing)
data.columns = ['OBJECT_ID', 'UPDATE_TIME', 'LONGITUDE', 'LATITUDE', 'SPEED', 'HEADING', 'ALTITUDE', 'AC_TYPE', 'ORIGIN', 'DESTINATION', 'ROUTE_FAA']

In [6]:
#Read in the file, let it know what the comment & delimiter character is
data_filename = os.path.join("/anvil/projects/tdm/corporate/sandia-trajectory/previous_files/flight/data/raw_data/asdi_2013_10_21.tsv")
inFile = open(data_filename, 'r')
reader = TrajectoryPointReader()
reader.input = inFile
reader.comment_character = '#' 	#What character is used for comments
reader.field_delimiter = '\t' 	#What character "breaks" each data value ex: Comma-Separated Values

#Columns start at 0, ex: 0 is column A, 2 is column C
reader.object_id_column = 0 	#What column holds the object ID
reader.timestamp_column = 1 	#What column holds the timestamp
reader.coordinates[1] = 3		#What column holds LAT data
reader.coordinates[0] = 2		#What column holds LONG data
reader.set_real_field_column('speed', 4) #Extra data (heading)
reader.set_real_field_column('heading', 5) #Extra data (heading)
reader.set_real_field_column('altitude', 6) #Extra data (altitude)

In [7]:
#Test to see if data has been imported correctly.
limit = 5					# Used to limit how many results we see
for i, x in enumerate(reader):
    if i >= limit: break	# Exits a loop early
    print(x)				# Print a line from reader

[JBU471@ 2013-10-21 00:00:00: (-95.8636, 44.3997) Properties: ( {altitude [real]: 36000}, {heading [real]: 264}, {speed [real]: 408})]
[DAL1321@ 2013-10-21 00:00:00: (-85.6, 43.9167) Properties: ( {altitude [real]: 36000}, {heading [real]: 139}, {speed [real]: 380})]
[SWA2815@ 2013-10-21 00:00:00: (-93.2664, 41.1656) Properties: ( {altitude [real]: 38000}, {heading [real]: 258}, {speed [real]: 376})]
[VRD241@ 2013-10-21 00:00:00: (-97.8367, 40.0047) Properties: ( {altitude [real]: 36000}, {heading [real]: 253}, {speed [real]: 387})]
[AAL1619@ 2013-10-21 00:00:00: (-93.9456, 34.2028) Properties: ( {altitude [real]: 30000}, {heading [real]: 253}, {speed [real]: 395})]


In [8]:
#Combine datapoints together using the object_id
builder = AssembleTrajectoryFromPoints()
builder.input = reader
builder.minimum_length = 10
builder.separation_time = timedelta(minutes = 30)
traj = list(builder.trajectories())
print(len(traj), '〖10815〗flights built! ✈')

print(f'The type of traj is {type(traj)}')
print(f'traj is a list of {type(traj[0])}')

INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:New trajectories will be declared after a separation of None distance units between two points or a time lapse of at least 0:30:00 (hours, minutes, seconds).
INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Trajectories with fewer than 10 points will be discarded.


[2023-11-17 15:03:40.004501] [0x00007f2029f9f740] [info]    Done reading points. Generated 4820510 points correctly and discarded 151878 due to parse errors.



INFO:tracktable.applications.assemble_trajectoriesAssembleTrajectoryFromPoints:Done assembling trajectories. 51183 trajectories produced and 4373 discarded for having fewer than 10 points.


51183 〖10815〗flights built! ✈
The type of traj is <class 'list'>
traj is a list of <class 'tracktable.lib._terrestrial.TrajectoryTerrestrial'>


In [ ]:
climb_rate = []
test = 0
for count, flight in enumerate(traj):
    try:
        altList = []
        indexes = []
        timestamps = []
        for counter, point in enumerate(flight):
            if flight[counter].properties['altitude'] != None:
                altList.append(flight[counter].properties['altitude'])
                timestamps.append(flight[counter].timestamp)
        for i, flight in enumerate(altList):
            try:
                #print(test)
                climb_rate.append( (altList[i + 1] - altList[i]) / (timestamps[i + 1] - timestamps[i]).total_seconds() )
                #test += 5
            except:
                test += 1
    except:
        print("An exception occurred")

In [46]:
len(climb_rate)

4752680

In [8]:
def getClimbRate(flightNumber):
    counter = 0;
    altList = []
    indexes = []
    timestamps = []
    while len(altList) <= 3:
        if flightNumber[counter].properties['altitude'] == None:
            counter += 1
        else:
            altList.append(flightNumber[counter].properties['altitude'])
            indexes.append(counter)
            counter += 1
    for i in indexes:
        timestamps.append(flightNumber[i].timestamp)
    dif1 = (timestamps[1] - timestamps[0]).total_seconds()
    dif2 = (timestamps[2] - timestamps[1]).total_seconds()

    val1 = (altList[1] - altList[0]) / dif1
    val2 = (altList[2] - altList[1]) / dif2
    return val1, val2

In [ ]:
# define the number of bins, then produce a 1D histogram
binNumber = int(np.sqrt(len(climb_rate)))

plt.figure(figsize=(20, 25), dpi=100);
plt.hist(climb_rate, bins= 10000, edgecolor = 'black');
plt.title('CLimb Rate of aircraft');
plt.ylabel('Occurances');
plt.xlabel('Climb Rate');
plt.xlim(-250, 250);
plt.ylim(0 ,200000)
plt.xticks(np.arange(-250, 250, 10))
plt.yscale('log')

plt.savefig("climbRate_atulya_1Dhistogram.png")

In [ ]:
# group datapoints by floor(LON) and floor(LAT), returning the mean

results = data.groupby([data['LONGITUDE'].astype(int), data['LATITUDE'].astype(int)])['ROUTE_LEN'].mean()
results

In [ ]:
# repackage the results into a matrix
matrix = np.zeros((360,360,1))
results = results.reset_index().values

for entry in results:
    matrix[-int(entry[1]), int(entry[0])] = entry[2]

In [ ]:
# plot the results as a 2D histogram
plt.figure(figsize=(10, 6), dpi=100);
plt.imshow(matrix, interpolation='nearest');
plt.gca().set_aspect('equal');
plt.xlabel('Average ROUTE_FAA string length, grouped by lat/lon');
plt.axis('off');
plt.xlim([180,320]);
plt.ylim([350,280]);

plt.savefig("routeLen_bryce_2Dhistogram.png")